In [1]:
#IMPORT MODULES
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

from keras.models import Sequential, load_model
from keras.layers import Dense

import numpy as np
import pandas as pd
import spacy


/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: Deprecation

In [2]:
#INIITIALISATIONS
nlp = spacy.load('en_core_web_sm')
sid = SentimentIntensityAnalyzer()
df = pd.read_csv('amazon_reviews.tsv', sep = '\t')
df1 = df.sample(n=1000, random_state=42)  

In [44]:
df1.columns

Index(['eDOC_ID', 'LABEL', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY',
       'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT',
       'POS_COUNTS', 'NUM NOUNS', 'NUM VERBS', 'NUM ADJECTIVES', 'NUM ADVERBS',
       'REVIEW_LENGTH', 'SENTIMENT SCORE', 'MAX_SIMILARITY', 'NUM_REVIEWS',
       'NUM_ORGS', 'AVERAGE_RATING', 'RATING_DEVIATION', 'cluster_label'],
      dtype='object')

In [3]:
#FUNCTIONS TO COUNT POS
# Function to count nouns in a text
def count_nouns(Pos_counts):
    noun_count = Pos_counts.get(spacy.parts_of_speech.NOUN, 0)
    return noun_count

# Function to count verbs in a text
def count_verbs(Pos_counts):
    verb_count = Pos_counts.get(spacy.parts_of_speech.VERB, 0)
    return verb_count

# Function to count adjectives in a text
def count_adjectives(Pos_counts):
    adjective_count = Pos_counts.get(spacy.parts_of_speech.ADJ, 0)
    return adjective_count

# Function to count adverbs in a text
def count_adverbs(Pos_counts):
    adverb_count = Pos_counts.get(spacy.parts_of_speech.ADV, 0)
    return adverb_count

def pos_counts(text):
    doc = nlp(text)
    Pos_counts = doc.count_by(spacy.attrs.POS)
    return Pos_counts

In [5]:
#ADD MAXIMUM SIMILARITY COLUMN

#INITIALISE
tfidfvectoriser = TfidfVectorizer()
tfidf_matrix = tfidfvectoriser.fit_transform(df1['REVIEW_TEXT'])

#CREATE COSINE SIMILARITY MATRIX
cosine_similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

#FIND MAX SIMILARITIES
max_similarities = []
for i, row in enumerate(cosine_similarity_matrix):
    #ALL OTHER BUT WITH ITSELF [ALWAYS 1]
    max_similarity = max(row[:i].tolist() + row[i+1:].tolist())
    max_similarities.append(max_similarity)
    

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


In [6]:
df1['MAX_SIMILARITY'] = max_similarities

In [45]:
#X -> FEATURE DATA
X = df1[['REVIEW_TEXT','RATING', 'VERIFIED_PURCHASE', 'NUM NOUNS', 'NUM ADVERBS', 'NUM ADJECTIVES', 'NUM VERBS', 'REVIEW_LENGTH', 'SENTIMENT SCORE', 'MAX_SIMILARITY','RATING_DEVIATION','NUM_ORGS']]


#Y -> LABELS
Y = df1['LABEL']

In [46]:

#PIPELINE FOR VECTORISATION OF TEXT AND SCALING OF NUMBERS
text_pipeline = Pipeline([
    ('selector', FunctionTransformer(
        lambda x: x['REVIEW_TEXT'], validate=False)),
    ('tfidf', TfidfVectorizer()),
])
numeric_pipeline = Pipeline([
    ('selector', FunctionTransformer(
        lambda x: x[['NUM NOUNS','NUM VERBS','NUM ADVERBS','NUM ADJECTIVES','VERIFIED_PURCHASE', 'RATING', 'REVIEW_LENGTH', 'SENTIMENT SCORE']], validate=False)),
    ('scaler', StandardScaler()),
])

feature_union = FeatureUnion([
    ('text_features', text_pipeline),
    ('numeric_features', numeric_pipeline)
])

In [47]:
#TRANSFORM X
X = feature_union.fit_transform(X)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [48]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

#NUMBER OF CLUSTERS
n_clusters = 5

#CREATE AND FIT THE K-MEANS MODEL
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpand

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpand

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/kpand

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)

In [52]:
#GET THE CLUSTER LABELS
cluster_labels = kmeans.labels_
df1['cluster_label'] = cluster_labels

df1['cluster_label'].value_counts()

0    443
3    321
2    187
1     44
4      5
Name: cluster_label, dtype: int64

In [53]:
from sklearn import metrics

# Extract true labels and cluster labels
true_labels = df1['LABEL']
cluster_labels = df1['cluster_label']

# Calculate evaluation metrics
accuracy = metrics.accuracy_score(true_labels, cluster_labels)
adjusted_rand_index = metrics.adjusted_rand_score(true_labels, cluster_labels)
silhouette_score = metrics.silhouette_score(X_scaled, cluster_labels)

print("Accuracy:", accuracy)
print("Adjusted Rand Index:", adjusted_rand_index)
print("Silhouette Score:", silhouette_score)

Accuracy: 0.0
Adjusted Rand Index: 0.18826957195930055
Silhouette Score: 0.26032409013515195


/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:124: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; f

In [55]:
# Group the DataFrame by cluster_label and calculate the counts for each label
label_counts = df1.groupby('cluster_label')['LABEL'].value_counts().unstack(fill_value=0)

# Extract the counts for "Fake" and "Not Fake"
fake_counts = label_counts['Fake']
not_fake_counts = label_counts['Not Fake']

print("Fake Counts:")
print(fake_counts)
print("\nNot Fake Counts:")
print(not_fake_counts)

Fake Counts:
cluster_label
0    120
1      7
2     99
3    275
4      1
Name: Fake, dtype: int64

Not Fake Counts:
cluster_label
0    323
1     37
2     88
3     46
4      4
Name: Not Fake, dtype: int64


In [50]:
df1[['cluster_label','LABEL']]

,cluster_label,LABEL
2312,3,Fake
16845,0,Not Fake
6105,3,Fake
18082,2,Not Fake
11335,2,Not Fake
17677,0,Not Fake
1092,3,Fake
8364,3,Fake
18550,2,Not Fake
11423,0,Not Fake


In [27]:
Y = df1['LABEL'].apply(lambda d: 1 if 'Fake' else 0)

In [37]:
df1['LABEL']

2312         Fake
16845    Not Fake
6105         Fake
18082    Not Fake
11335    Not Fake
17677    Not Fake
1092         Fake
8364         Fake
18550    Not Fake
11423    Not Fake
6757         Fake
12893    Not Fake
17446    Not Fake
11823    Not Fake
4772         Fake
18882    Not Fake
7700         Fake
5018         Fake
1474         Fake
4966         Fake
6241         Fake
474          Fake
19606    Not Fake
3316         Fake
2714         Fake
8918         Fake
5701         Fake
5948         Fake
4059         Fake
19103    Not Fake
           ...   
6595         Fake
4309         Fake
3958         Fake
19404    Not Fake
16400    Not Fake
1272         Fake
3189         Fake
13293    Not Fake
20497    Not Fake
8887         Fake
1475         Fake
2533         Fake
13674    Not Fake
12004    Not Fake
14480    Not Fake
3836         Fake
13188    Not Fake
3472         Fake
5735         Fake
16532    Not Fake
20188    Not Fake
606          Fake
17601    Not Fake
2826         Fake
13925    N

In [33]:
print(classification_report(Y,Y1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.96      0.98      1000

   micro avg       0.96      0.96      0.96      1000
   macro avg       0.50      0.48      0.49      1000
weighted avg       1.00      0.96      0.98      1000



/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [103]:
#TRAIN-TEST SPLIT
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [104]:
#TRAIN THE MODEL
classifier = LogisticRegression()
classifier.fit(X_TRAIN, Y_TRAIN)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [105]:
#PREDICTIONS
predictions = classifier.predict(X_TEST)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/linear_model/base.py:283: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [106]:
#CHECK PRECISION, RECALL, F1 SCORE
print(classification_report(Y_TEST, predictions))

              precision    recall  f1-score   support

        Fake       0.89      0.69      0.78        94
    Not Fake       0.77      0.92      0.84       106

   micro avg       0.81      0.81      0.81       200
   macro avg       0.83      0.81      0.81       200
weighted avg       0.83      0.81      0.81       200



In [15]:
#ADD AN AVERAGE-RATING COLUMN
average_ratings = df.groupby('PRODUCT_ID')['RATING'].mean()

df['AVERAGE_RATING'] = df['PRODUCT_ID'].map(average_ratings)

In [16]:
#ADD AN AVERAGE RATING COLUMN

#AVERAGE RATINGS

# Create a dictionary mapping product IDs to average ratings from df
average_ratings_dict = df.set_index('PRODUCT_ID')['AVERAGE_RATING'].to_dict()

df1['AVERAGE_RATING'] = df1['PRODUCT_ID'].map(average_ratings_dict)

In [17]:
#ADD A DEVIATION FROM AVERAGE RATING COLUMN
df1['RATING_DEVIATION'] = df1['RATING']-df1['AVERAGE_RATING']

df1[['RATING_DEVIATION']]

,RATING_DEVIATION
2312,0.0
16845,0.0
6105,0.0
18082,0.0
11335,0.0
17677,0.0
1092,0.0
8364,0.0
18550,0.0
11423,0.0


Honey Badger Garden Gloves for Digging & Planting - No More Worn Out Fingertips - Claws on BOTH Hands - Womens & Mens Unisex - PATENT PENDING                                                                                                                                                                                                                                                                       5
The Last of Us - PlayStation 3                                                                                                                                                                                                                                                                                                                                                                                      4
Anker 8200 DPI High Precision Programmable Laser Gaming Mouse for PC, 9 Programmable Buttons, Weight Tuning Cartridges, Omron Micro                                                         

In [18]:
#ADD A NUMBER OF REVIEWS COLUMN
num_reviews = df.groupby('PRODUCT_ID').size()

df1['NUM_REVIEWS'] = df1['PRODUCT_ID'].map(num_reviews)

In [19]:
df1['NUM_REVIEWS'].value_counts()

1    806
2    168
3     21
4      5
Name: NUM_REVIEWS, dtype: int64

In [20]:
#ADD A NUMBER OF TIMES ORGANISATIONS WERE MENTIONED COLUMN
def count_org(text):
    doc = nlp(text)
    org_count = len([ent.text for ent in doc.ents if ent.label_ == 'ORG'])
    return org_count

num_orgs = df1['REVIEW_TEXT'].apply(count_org)

In [12]:
df1 = df1.assign(NUM_ORGS=num_orgs)

In [13]:
df1['NUM_ORGS']

2312     0
16845    0
6105     0
18082    1
11335    0
17677    0
1092     0
8364     0
18550    0
11423    0
6757     0
12893    0
17446    0
11823    0
4772     0
18882    0
7700     0
5018     1
1474     0
4966     0
6241     1
474      0
19606    0
3316     0
2714     0
8918     1
5701     0
5948     0
4059     0
19103    0
        ..
6595     0
4309     1
3958     1
19404    0
16400    0
1272     0
3189     0
13293    0
20497    7
8887     0
1475     0
2533     0
13674    0
12004    2
14480    0
3836     0
13188    0
3472     1
5735     2
16532    1
20188    1
606      0
17601    0
2826     0
13925    0
1338     0
484      0
1719     0
19712    0
218      1
Name: NUM_ORGS, Length: 1000, dtype: int64

In [108]:
filtered_df = df1[(df1['LABEL'] == 'Fake') & (df1['RATING_DEVIATION'] > 0)]
num_records = filtered_df.shape[0]
num_records

31

In [111]:
df1['LABEL'].value_counts()

Fake        502
Not Fake    498
Name: LABEL, dtype: int64

In [115]:
#TRYING DEEP LEARNING

#->0 WHEN NON FAKE AND 1 WHEN FAKE
Y = df1['LABEL'].apply(lambda x: 1 if x == 'Fake' else 0)

#CONVERT LABELS TO ONE-HOT ENCODING
from keras.utils import to_categorical

Y = to_categorical(Y)

In [119]:
df1['VERIFIED_PURCHASE'].value_counts()

1    562
0    438
Name: VERIFIED_PURCHASE, dtype: int64

In [150]:
#X-> FEATURE DATA [NUMERICAL]
X = df1[['RATING','REVIEW_LENGTH', 'RATING_DEVIATION', 'SENTIMENT SCORE', 'VERIFIED_PURCHASE']]

#TRAIN-TEST SPLIT
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [123]:
#SCALE ALL VALUES
scaler_object = MinMaxScaler()
scaler_object.fit(X_TRAIN)

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [126]:
scaled_X_TRAIN = scaler_object.transform(X_TRAIN)
scaled_X_TEST = scaler_object.transform(X_TEST)

In [137]:
model = Sequential()

model.add(Dense(8, input_dim=5, activation='relu'))
model.add(Dense(8, input_dim=5, activation='relu'))
model.add(Dense(8, input_dim=5, activation='relu'))
model.add(Dense(8, input_dim=5, activation='relu'))
model.add(Dense(8, input_dim=5, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
#FIT MODEL ON SCALED DATA
model.fit(scaled_X_TRAIN, Y_TRAIN, epochs = 400, verbose = 2)

Epoch 1/400
25/25 - 0s - loss: 0.4505 - accuracy: 0.8037 - 38ms/epoch - 2ms/step
Epoch 2/400
25/25 - 0s - loss: 0.4515 - accuracy: 0.8037 - 36ms/epoch - 1ms/step
Epoch 3/400
25/25 - 0s - loss: 0.4489 - accuracy: 0.8037 - 35ms/epoch - 1ms/step
Epoch 4/400
25/25 - 0s - loss: 0.4507 - accuracy: 0.8037 - 38ms/epoch - 2ms/step
Epoch 5/400
25/25 - 0s - loss: 0.4497 - accuracy: 0.8000 - 36ms/epoch - 1ms/step
Epoch 6/400
25/25 - 0s - loss: 0.4493 - accuracy: 0.8012 - 34ms/epoch - 1ms/step
Epoch 7/400
25/25 - 0s - loss: 0.4488 - accuracy: 0.8000 - 36ms/epoch - 1ms/step
Epoch 8/400
25/25 - 0s - loss: 0.4483 - accuracy: 0.8050 - 35ms/epoch - 1ms/step
Epoch 9/400
25/25 - 0s - loss: 0.4471 - accuracy: 0.8037 - 37ms/epoch - 1ms/step
Epoch 10/400
25/25 - 0s - loss: 0.4498 - accuracy: 0.8000 - 37ms/epoch - 1ms/step
Epoch 11/400
25/25 - 0s - loss: 0.4477 - accuracy: 0.8037 - 37ms/epoch - 1ms/step
Epoch 12/400
25/25 - 0s - loss: 0.4469 - accuracy: 0.8037 - 38ms/epoch - 2ms/step
Epoch 13/400
25/25 - 0s -

25/25 - 0s - loss: 0.4477 - accuracy: 0.8087 - 30ms/epoch - 1ms/step
Epoch 102/400
25/25 - 0s - loss: 0.4462 - accuracy: 0.8050 - 30ms/epoch - 1ms/step
Epoch 103/400
25/25 - 0s - loss: 0.4465 - accuracy: 0.8050 - 31ms/epoch - 1ms/step
Epoch 104/400
25/25 - 0s - loss: 0.4441 - accuracy: 0.8025 - 31ms/epoch - 1ms/step
Epoch 105/400
25/25 - 0s - loss: 0.4462 - accuracy: 0.8062 - 30ms/epoch - 1ms/step
Epoch 106/400
25/25 - 0s - loss: 0.4442 - accuracy: 0.8012 - 31ms/epoch - 1ms/step
Epoch 107/400
25/25 - 0s - loss: 0.4468 - accuracy: 0.8025 - 33ms/epoch - 1ms/step
Epoch 108/400
25/25 - 0s - loss: 0.4433 - accuracy: 0.8050 - 30ms/epoch - 1ms/step
Epoch 109/400
25/25 - 0s - loss: 0.4442 - accuracy: 0.8062 - 31ms/epoch - 1ms/step
Epoch 110/400
25/25 - 0s - loss: 0.4413 - accuracy: 0.8050 - 32ms/epoch - 1ms/step
Epoch 111/400
25/25 - 0s - loss: 0.4438 - accuracy: 0.8037 - 32ms/epoch - 1ms/step
Epoch 112/400
25/25 - 0s - loss: 0.4428 - accuracy: 0.8037 - 32ms/epoch - 1ms/step
Epoch 113/400
25/2

Epoch 200/400
25/25 - 0s - loss: 0.4386 - accuracy: 0.8050 - 36ms/epoch - 1ms/step
Epoch 201/400
25/25 - 0s - loss: 0.4381 - accuracy: 0.8037 - 34ms/epoch - 1ms/step
Epoch 202/400
25/25 - 0s - loss: 0.4397 - accuracy: 0.8062 - 34ms/epoch - 1ms/step
Epoch 203/400
25/25 - 0s - loss: 0.4380 - accuracy: 0.8050 - 32ms/epoch - 1ms/step
Epoch 204/400
25/25 - 0s - loss: 0.4374 - accuracy: 0.8050 - 33ms/epoch - 1ms/step
Epoch 205/400
25/25 - 0s - loss: 0.4388 - accuracy: 0.8075 - 33ms/epoch - 1ms/step
Epoch 206/400
25/25 - 0s - loss: 0.4387 - accuracy: 0.8025 - 35ms/epoch - 1ms/step
Epoch 207/400
25/25 - 0s - loss: 0.4389 - accuracy: 0.8087 - 36ms/epoch - 1ms/step
Epoch 208/400
25/25 - 0s - loss: 0.4409 - accuracy: 0.8075 - 36ms/epoch - 1ms/step
Epoch 209/400
25/25 - 0s - loss: 0.4364 - accuracy: 0.8050 - 36ms/epoch - 1ms/step
Epoch 210/400
25/25 - 0s - loss: 0.4398 - accuracy: 0.8050 - 39ms/epoch - 2ms/step
Epoch 211/400
25/25 - 0s - loss: 0.4437 - accuracy: 0.8087 - 36ms/epoch - 1ms/step
Epoc

Epoch 299/400
25/25 - 0s - loss: 0.4341 - accuracy: 0.8087 - 36ms/epoch - 1ms/step
Epoch 300/400
25/25 - 0s - loss: 0.4372 - accuracy: 0.8037 - 36ms/epoch - 1ms/step
Epoch 301/400
25/25 - 0s - loss: 0.4335 - accuracy: 0.8087 - 34ms/epoch - 1ms/step
Epoch 302/400
25/25 - 0s - loss: 0.4303 - accuracy: 0.8050 - 32ms/epoch - 1ms/step
Epoch 303/400
25/25 - 0s - loss: 0.4330 - accuracy: 0.8100 - 32ms/epoch - 1ms/step
Epoch 304/400
25/25 - 0s - loss: 0.4329 - accuracy: 0.8087 - 31ms/epoch - 1ms/step
Epoch 305/400
25/25 - 0s - loss: 0.4329 - accuracy: 0.8087 - 32ms/epoch - 1ms/step
Epoch 306/400
25/25 - 0s - loss: 0.4342 - accuracy: 0.8062 - 32ms/epoch - 1ms/step
Epoch 307/400
25/25 - 0s - loss: 0.4322 - accuracy: 0.8037 - 32ms/epoch - 1ms/step
Epoch 308/400
25/25 - 0s - loss: 0.4331 - accuracy: 0.8125 - 31ms/epoch - 1ms/step
Epoch 309/400
25/25 - 0s - loss: 0.4320 - accuracy: 0.8087 - 30ms/epoch - 1ms/step
Epoch 310/400
25/25 - 0s - loss: 0.4325 - accuracy: 0.8087 - 30ms/epoch - 1ms/step
Epoc

Epoch 398/400
25/25 - 0s - loss: 0.4289 - accuracy: 0.8100 - 40ms/epoch - 2ms/step
Epoch 399/400
25/25 - 0s - loss: 0.4319 - accuracy: 0.8087 - 37ms/epoch - 1ms/step
Epoch 400/400
25/25 - 0s - loss: 0.4319 - accuracy: 0.8112 - 40ms/epoch - 2ms/step


In [144]:
predictions = np.argmax(model.predict(scaled_X_TEST),axis=1)

7/7 [==============================] - 0s 1ms/step


In [152]:
Y_TEST = Y_TEST.argmax(axis=1)

In [153]:
print(classification_report(Y_TEST, predictions))

              precision    recall  f1-score   support

           0       0.74      0.90      0.81       106
           1       0.85      0.65      0.73        94

   micro avg       0.78      0.78      0.78       200
   macro avg       0.79      0.77      0.77       200
weighted avg       0.79      0.78      0.78       200



In [143]:
model.save('unsupervised_model.h5')